# 图片处理
## 使用PIL读取图片，使用torchvision.transforms处理图片
reference:https://pytorch.org/docs/stable/torchvision/transforms.html#torchvision-transforms

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import torchvision
import torchvision.transforms as transforms
import sys

## 使用PIL打开图片并使用matplotlib把图片展示出来

In [ ]:
img = Image.open('scene.jpeg').convert('RGB') # 加上convert防止有些4通道图片导致问题
# 我使用的图片是H*W=1200*1920的RGB3通道图片
print(type(img))
print(img.size)

plt.imshow(img)

## 使用array的形式展示图片，可以看到图片的shape是1920×1080，RGB三通道

In [ ]:
img_array = np.array(img)
print(img_array.shape)
plt.imshow(img_array)

## 使用PIL的resize可以很方便的改变图片的shape

In [ ]:
img_resized = img.resize((224,224))
plt.imshow(img_resized)

## 使用transform的CenterCrop类可以实现从图片中心进行切割
参数为裁剪尺寸的tuple

In [ ]:
cenercrop = transforms.CenterCrop(size=(224,224)) # 参数也可以是整数,表示size为int*int的正方形裁剪
img_1 = cenercrop(img)
plt.imshow(img_1)

## Grayscale:Convert image to grayscale. 将图片转换成灰度图
参数设置为3时打印为灰色，设置为1不是灰色，没有搞懂<br>
Grayscale version of the input. - If num_output_channels == 1 : returned image is single channel - If num_output_channels == 3 : returned image is 3 channel with r == g == b

In [ ]:
grayscale = transforms.Grayscale(num_output_channels=3)
img_2 = grayscale(img)
plt.imshow(img_2)

打印灰度图和原图的一些像素值，发现发生了改变

In [ ]:
gray_array = np.array(img_2)
print(gray_array[600,900:1000,0])
print(img_array[600,900:1000,0])

## Pad:Pad the given PIL Image on all sides with the given “pad” value. 在图片周围填充
处理对象是PIL image. <br>
在图片周围填充第一个参数控制填充参数填充，第二个参数控制填充的像素值

In [ ]:
padding = transforms.Pad(padding=10, fill=(0,0,0))
img_3 = padding(img)
plt.imshow(img_3)
print(img_3.size)

pad_array = np.array(img_3)
print(pad_array[0,:10,:].reshape(1,-1))

## Resize:Resize the input PIL Image to the given size.  改变图片大小
处理对象是PIL image. <br>
第一个参数为size，第二个参数为插值方法，默认使用双线性插值. <br>
区别于crop：crop为裁剪，resize为把图片变形.

In [ ]:
resize = transforms.Resize((200,200), interpolation=2)
img_4 = resize(img)
plt.imshow(img_4)
print(img_4.size)

## RandomHorizontalFlip:Horizontally flip the given PIL Image randomly with a given probability.  随机水平翻转图片
处理对象是PIL image. <br>
参数p为翻转概率，这里为了看到翻转图片所以设置为1，使用时可以使用0.5或其他值. <br>
RandomVerticalFlip(p=0.5) 随机垂直翻转图片，使用方法相同

In [ ]:
hflip = transforms.RandomHorizontalFlip(p=1)
img_5 = hflip(img)
plt.imshow(img_5)

## RandomRotation: Rotate the image by angle. 以随机角度旋转图片
处理对象是PIL image. <br>
图片旋转的角度从第一个参数中随机选择，如果输入为一个数字，则从（-degree,+degree)中随机选择. <br>
expend=True则会改变图片尺寸来使得整张图片能被容纳.

In [ ]:
rotate = transforms.RandomRotation(degrees=(-30,30), resample=False, expand=False, center=None)
img_6 = rotate(img)
plt.imshow(img_6)

## RandomCrop: Crop the given PIL Image at a random location. 随机位置裁剪图片
size:裁剪后尺寸 <br>
padding:四周填充像素宽度 <br>
fill: 填充像素值 <br>

In [ ]:
randCrop = transforms.RandomCrop(size=(1200,1920), padding=100, pad_if_needed=False, 
                                 fill=(0,0,0), padding_mode='constant')
img_7 = randCrop(img)
plt.imshow(img_7)

## ToTensor: Convert a PIL Image or numpy.ndarray to tensor. 将图片转换成tensor

Converts a PIL Image or numpy.ndarray (H x W x C) in the range [0, 255] to a torch.FloatTensor of shape (C x H x W) in the range [0.0, 1.0] if the PIL Image belongs to one of the modes (L, LA, P, I, F, RGB, YCbCr, RGBA, CMYK, 1) or if the numpy.ndarray has dtype = np.uint8 <br><br>
处理对象是PIL image. <br>

1.PIL或者ndarray通道数在后，转化成tensor后通道数在前; <br>
2.数据范围转换为[0.0,1.0].

In [ ]:
to_tensor = transforms.ToTensor()
img_tensor = to_tensor(img)
print(type(img_tensor))
print(img_tensor.size())
print('{} in ndarray and {} in tensor'.format(img_array[500,500,0],img_tensor[0,500,500])) # 135 = 255 * 0.529

## Normalize: Normalize a tensor image with mean & standard deviation.tensor归一化处理
Normalize a tensor image with mean and standard deviation. <br>
input[channel] = (input[channel] - mean[channel]) / std[channel] <br>
处理对象是tensor. <br>


In [ ]:
normalize = transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5), inplace=False)
# mean和std都设为0.5相当于是把数据从[0.0,1.0]放缩到[-1.0,1.0]
t = normalize(img_tensor)
print('{} before normalize and {} after'.format(img_tensor[0,500,500],t[0,500,500]))

## Compose:Composes several transforms together.  集合transform操作
Parameters：transforms (list of Transform objects) – list of transforms to compose. <br>
按照列表给出的操作顺序依次进行transform操作

In [ ]:
compose = transforms.Compose([
    transforms.Pad(padding=100, fill=(0,0,0)),
    transforms.RandomHorizontalFlip(p=1),
    transforms.Resize((200,200), interpolation=2)   
])
img_8 = compose(img)
plt.imshow(img_8)